In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# loading data
df = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/results.csv')
print(df.shape)
df.head()

In [ ]:
# check for null values
df.isna().sum()

In [ ]:
# remove null values
df.dropna(inplace=True)
print(df.isna().sum().sum())

In [ ]:
# basic info about the dataframe
df.info()

In [ ]:
# teams that played the number of away matches
df.away_team.value_counts()

In [ ]:
tournaments = df.tournament.unique().tolist()
print((f'There are {len(tournaments)} tournaments in this dataset'))

In [ ]:
# most games are played in these cities 
df.city.value_counts()[:10]

In [ ]:
# parsing dates
df['date'] = pd.to_datetime(df['date'],errors='coerce',format='%Y-%m-%d')

df['Year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
df['weekday'] = df['date'].map(lambda x: day_order[x.weekday()] )

print(df.dtypes)
df.head()

In [ ]:
# making a new column to denote a 'win' ,'lose' or 'tie'
def find_result(df):
    if df.home_score > df.away_score:
        return 'win'
    if df.away_score > df.home_score:
        return 'lost'
    else:
        return 'tie'
    
df['result'] = df.apply(find_result,axis=1)
df

# EDA

### 1) In which month most matches are played ?

In [ ]:
fig = px.histogram(df,
                   x="month",
                   text_auto=True,
                   title='Number of matches played per month',
                   category_orders= {"month": list(range(1,13))})
fig.update_layout(bargap=0.2)
fig.update_xaxes(type='category',title_text='Number of months')
fig.update_yaxes(title_text='Matches played')
fig.show()

On June , august, september most matches are played

### 2) On which weekdays most matches are played ?

In [ ]:
fig = px.histogram(df,
                   x="weekday",
                   text_auto=True,
                   title='Number of matches played per weekday',
                   category_orders= {"weekday": day_order}
                   )
fig.update_layout(bargap=0.2)
fig.update_xaxes(type='category',title_text='Number of months')
fig.update_yaxes(title_text='matches played')
fig.show()

On wednesday , sunday most matches are played

### 3) Does home advantage helps a team to win ?

In [ ]:
fig = go.Figure()
fig = px.histogram(df,
           x = 'neutral',
           color='result',
           title='Venue distribution on basis of result',
           barmode='group',
           text_auto=True,
           )
fig.show()

From the above plots if we only see the win count in **false**(not a neutral venue or home ground) and **true**(neutral ground) then there is home advantage as in **false**(***win is around 17K***) and in **true**(***win = 4807***) 

### 4) Number of matches played in tournaments each year ?

In [ ]:
tournaments_count_over_years = pd.DataFrame(df.groupby(['Year','tournament'])['tournament'].count()).rename(columns={'tournament':'matches_played'})
tournaments_count_over_years = tournaments_count_over_years.reset_index().sort_values('Year',ascending=False)

fig = go.Figure()
fig = px.line(tournaments_count_over_years,
              x="Year", y=["matches_played"],
              color='tournament',
              labels={'matches_played':'number of matches'})

fig.update_layout(title='Number of matches played in each tournament over the years',
                  yaxis_title='Matches played')

fig.show()

There are **609** ***FIFA World Cup qualification*** matches played in 2021 highest till now. It is followed by 2001 ***491 FIFA World Cup qualification***  matches played and then **426** ***Friendly*** matches played in 2018.

### 5) Who is the best team of all time ?

There home_teams and away_team available . So to find the best team let's find the best home_teams and away_teams then compare both.

In [ ]:
best_home_team = df[df.home_score > df.away_score]
best_home_team = pd.DataFrame(best_home_team.groupby(['home_team'])['home_team'].agg('count'))
best_home_team = best_home_team.rename(columns={'home_team':'home_matches_won'}).sort_values('home_matches_won',ascending=False).reset_index()
best_home_team[:5]

In [ ]:
best_away_team = df[df.home_score < df.away_score]
best_away_team = pd.DataFrame(best_away_team.groupby(['away_team'])['away_team'].agg('count'))
best_away_team = best_away_team.rename(columns={'away_team':'away_matches_won'}).sort_values('away_matches_won',ascending=False).reset_index()
best_away_team[:5]

By comapring the top 5 home teams and away teams we can say that Brazil has won most number of home matches. As we seen before there is always home advantage as a home team playes well in it's home ground. 

### 6) Which teams dominated different eras of football ?

In [ ]:
# making a new column just for the purpose of analysis
def find_era(df):
    eighteens = np.arange(1872,1900).tolist()
    nineteens = np.arange(1900,2000).tolist()
    twenties = np.arange(2000,3000).tolist()
    
    if df.Year in eighteens:
        return '18s'
    if df.Year in nineteens:
        return '19s'
    if df.Year in twenties:
        return '20s'

df['era'] = df.apply(find_era,axis=1)
df.sample(5)

In [ ]:
# Most TROPHIES won in each tournament by each team
# we will focus on best home teams

best_teams = df[(df.result=='win')]
best_teams = best_teams.groupby(['era','home_team'])[['home_score']].count().sort_values('home_score',ascending=False).rename(columns={'home_score':'matches_won'}).reset_index()
best_teams[:20]

In [ ]:
fig = px.bar(best_teams, x="home_team",
             y = 'matches_won' ,
             color='era',
             title='Total matches won by teams in each era'
             )
fig.show()

We can see that in the **19s** era(or from 1900-2000) **Brazil** grabbed the top spot followed by ***Argentina , Sweden , Germany***. Many legends played in that era like from **Brazil** we had ***Pele,Garrincha,Ronaldo*** etc. while from **Argentina** we had ***Diego Maradona,Alfredo di Stefano,Gabriel Batistuta***,etc. Also advancements in televisions can be a reason for increasing footbal's popularity. 

I want to find which are the best teams in world's top world tournaments

In [ ]:
top_tours = df[
   (df.tournament == 'FIFA World Cup') |
   (df.tournament == 'UEFA Nations League')|
   (df.tournament == 'Copa América')|
   (df.tournament == 'UEFA Euro')|
   (df.tournament == 'African Cup of Nations')|
   (df.tournament == 'Confederations Cup')|
   (df.tournament == "King's Cup")|
   (df.tournament == 'AFC Asian Cup')
  ]

top_tours

In [ ]:
winning_home_teams = top_tours[(top_tours.result == 'win')].groupby(['tournament','home_team'])[['home_team']].count().rename(columns={'home_team':'total_matches_won'})

rows = 8
fig = make_subplots(
    rows=rows, cols=1,
    subplot_titles=("AFC Asian Cup", "African Cup of Nations", "Confederations Cup", "Copa América",'FIFA World Cup',"King's Cup",'UEFA Nations League','UEFA Euro'))


cups = ["AFC Asian Cup", "African Cup of Nations", "Confederations Cup", "Copa América",'FIFA World Cup',"King's Cup",'UEFA Nations League','UEFA Euro']

# plot the top 10 teams to won most number of matches in a tournament
for idx,cup in enumerate(cups):
    fig.add_trace(go.Bar(
                     x=winning_home_teams.loc[cup].sort_values('total_matches_won',ascending=False)[:10].index, 
                     y=winning_home_teams.loc[cup].sort_values('total_matches_won',ascending=False)[:10].total_matches_won,
                     name=cup,
                     showlegend=False,),
                     row=idx+1, col=1)

fig.update_layout(height=1100,
                  width=1300,
                  title_text="Top 10 Home Teams winning most matches in particular tournament")

fig.show()

### 7) What countries play with each other the most ?

In [ ]:
most_matches_together = df.groupby(['home_team','away_team'])[['home_team']].agg('count').rename(columns={'home_team' : 'matches_played_together'}).sort_values('matches_played_together',ascending=False).reset_index()
most_matches_together[:10]

**Argentina** and **Uruguay** have grabbed the 1st spot with ***179*** total matches together followed by **Austria** and **Hungry** with ***137*** matches together.

### 8) Which countries host the most matches where they themselves are not participating in ?

In [ ]:
not_participated_matches = df[(df.neutral==True)]['country'].value_counts().reset_index().rename(columns={'index':'country',
                                                                               'country':'matches_hosted'})

fig = px.bar(not_participated_matches[:50],
           x = 'country',
           y = 'matches_hosted',
           color='country',
           title='Top 50 countries that hosted matches in which it not participated',
           text_auto=True,
           )
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

fig.show()

**US** has hosted 894 matches in which it not participated. It is followed by **Malaysia**(473 matches) and **Quatar**(372 matches). 

### 9) Which teams are the most active in playing friendlies and friendly tournaments ?

In [ ]:
friendly_matches = df[df.tournament == 'Friendly'].groupby(['home_team','away_team'])[['home_team']].agg('count').rename(columns={'home_team':'friendly_matches_together'}).sort_values('friendly_matches_together',ascending=False).reset_index()
friendly_matches[:10]

**Austria** and **Hungary** have played most friendly matches together ***115*** matches followed by **Netherlands** and **Belgium** ***109*** matches.

## Goalscorers dataframe

In [ ]:
goalscorers = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/goalscorers.csv')
goalscorers.sample(5)

In [ ]:
goalscorers.info()

In [ ]:
most_goal_players = pd.DataFrame(goalscorers.groupby(['team','scorer'])[['scorer','minute']].agg({'scorer':'count',
                                                                     'minute':'mean'})).rename(columns={'scorer':'goals_scored' , 'minute':'avg_minute'}).sort_values('goals_scored',ascending=False).reset_index()[:10]

most_goal_players

**Cristiano Ronaldo** scored most goals(***91***) and has a average goal scoring time of 54 min(or mostly scores in 2nd half of the game). He is followed by **Robert Lewandowski** (***56 goals***) and average time 51 min(mostly scores in 2nd half) and **Lionel Messi**(***54 goals***) and average time 46 min(or we can say mostly scores at the end of 1st half)

### 10) Which player has scored more goals in penalty ?

In [ ]:
penalty_goals = goalscorers[goalscorers['penalty'] == True]
penalty_goals.groupby(['team','scorer'])[['scorer']].agg({'scorer':'count',
                                                         }).rename(columns={'scorer':'goals_scored' }).sort_values('goals_scored',ascending=False).reset_index()[:10]

Again 1st spot is taken by **Cristiano Ronaldo** scored ***16*** goals followed by **Lionel Messi** - ***14*** goals and **Hristo Stoichkov** - ***13*** goals.

In [ ]:
# parsing date 
goalscorers['date'] = pd.to_datetime(goalscorers['date'],errors='coerce',format='%Y-%m-%d')
goalscorers['Year'] = goalscorers['date'].dt.year

In [ ]:
# which teams scored their own goals
own_goals = goalscorers[goalscorers['own_goal'] == True]
own_goal_teams = own_goals.groupby(['Year','team'])[['own_goal']].agg('count').sort_values('own_goal',ascending=False).rename(columns={'own_goal':'times_own_goal_scored'}).reset_index()

fig = px.line(own_goal_teams, x='Year', y=['times_own_goal_scored'],
              color='team',hover_name='team',
              markers = True,
              title="Teams that scored own goals through the years")

fig.update_xaxes(tickvals=np.arange(1917,2023,5),tickangle=45, tickfont=dict(family='Rockwell', size=14))
fig.show()

**Germany** has most own goals(4) followed by **Russia,Crotia,Mexico,Spain,Canada**.  

### 11) Which team or player score most goals in 1st half or second half ?

In [ ]:
# dividing minute into 1st and 2nd half 
goalscorers['half'] = goalscorers['minute'].apply(lambda x: 'first_half' if (x < 45.0) else 'second_half')
goalscorers

In [ ]:
team_goal_in_halfs = goalscorers.groupby(['team','half'])[['scorer']].agg('count').sort_values('scorer',ascending=False).reset_index().rename(columns={'scorer':'goals_scored'})

fig = px.bar(team_goal_in_halfs[:50],
             x='team', 
             y='goals_scored',
             color="half",
             title="Most goals scored in first and second halfs by teams(starting from highest)",
             text_auto=True)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

Top 5 spots shows that **Brazil,Germany,Argentina,Spain,Netherlands** had scored most goals in **2nd half** of the match because there are often player replacement or change of playing technique or goal chasing after the 1st half. 

In [ ]:
player_goal_scored_in_halfs = pd.DataFrame(goalscorers.groupby(['scorer','half'])[['scorer']].agg('count')).rename(columns={'scorer':'goal_scored'}).sort_values('goal_scored',ascending=False).reset_index()

fig = px.bar(player_goal_scored_in_halfs[:50],
             x='scorer', 
             y='goal_scored',
             color="half",
             title="Most goals scored in first and second halfs by individual players(starting from highest)",
             text_auto=True)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

**Cristanio Ronaldo** is on 1st place and he mostly goals in second half.He is followed by **Ali Daei** and **Robert Lewandowski**. A point should be noted that these goals also include the penalty goals.